## Marketing Campaign

Solution by steps



### Step 0. Correct market simulation

The most important step!

In [1]:
import numpy as np
import networkx as nx
import pandas as pd
from tqdm import tqdm

In [2]:
edges = []
file = open('marketing_edges.txt')
for line in file.readlines():
    txt = line.split()
    ed = []
    ed.append(int(txt[0]))
    ed.append(int(txt[1]))
    edges.append(ed)
file.close()

In [3]:
G = nx.Graph()
G.add_nodes_from(np.arange(4039))
G.add_edges_from(edges)
A = nx.to_numpy_array(G)
deg = A.sum(axis=0)
n = 4039

In [5]:
# Main function

def simulate_market(strat, bablos=10000, A=A, deg=deg, n=4039):
    days_flag = True
    aff = np.zeros(n)
    for t in range(60):
        if t in strat:
            for adv in strat[t]:
                bablos -= 300 * deg[adv]
                aff[adv] = 1
            if bablos < 0:
                return -15000, 0, 0
        new_aff = (aff @ A * (1 - aff) > 0.18 * deg).astype('int')
        aff += new_aff
        num_new = new_aff.sum()
        bablos += 50 * num_new
        if num_new == 0:
            if days_flag:
                days_flag = False
            else:
                return bablos - 10000, t, aff.sum()
        else:
            days_flag = True
    return bablos - 10000, 60, aff.sum()

In [6]:
test_dict = {
    # On the first day, I will sign contracts with 47 and 138
    0: [47, 138],
    # On the third day, I will sign 2 more contracts
    2: [3789, 262]
}

# Oui, c'est vrai!
simulate_market(test_dict)

(-2350.0, 10, 29.0)

### Step 1. One in the field is not a warrior!



In [8]:
# Let's choose the best node.

n = 4039
best_strat = []

max_prof = 0
best_node = None
for i in tqdm(range(n)):
    test_strat = best_strat.copy()
    test_strat.append(i)
    prof = simulate_market({0: test_strat})[0]
    if prof > max_prof:
        max_prof = prof
        best_node = i
print(max_prof, best_node)

100%|██████████| 4039/4039 [00:56<00:00, 71.18it/s] 

30200.0 3057


In [10]:
# Check

simulate_market({0: [3057]})

(30200.0, 29, 701.0)

Wow! One small node was able to bring more profit than both baselines combined! 

In [ ]:
"""
Сложно!
Надо отдохнуть.
"""

### Step 2. Greedy algorithm

Let's implement a greedy algorithm. https://en.wikipedia.org/wiki/Greedy_algorithm

We will only add nodes on day zero in this step.

In [ ]:
# Maybe for a long time

n = 4039
best_strat = []
for j in range(10):
    max_prof = 0
    best_node = None
    for i in tqdm(range(n)):
        test_strat = best_strat.copy()
        test_strat.append(i)
        prof = simulate_market({0: test_strat})[0]
        if prof > max_prof:
            max_prof = prof
            best_node = i
    best_strat.append(best_node)
    print(j, best_strat, max_prof)

In [14]:
# Result

simulate_market({0: [3057, 3775, 2263, 3991, 443]})

(64000.0, 44, 1483.0)

64k. Not bad!

### Step 3. Advertising on credit

Let's implement a simplified function. We will assume that there is no restriction on the purchase of advertising. As if it could be taken on credit.

In fact, in the present model, we will simply buy part of the advertisement in the later days, when we receive money from the first advertisement.

In [15]:
def simple_simulate_market(aff, bablos=10000, A=A, deg=deg, n=4039):
    for i in range(n):
        if aff[i] == 1:
            bablos -= 300 * deg[i]
    for t in range(60):
        new_aff = (aff @ A * (1 - aff) > 0.18 * deg).astype('int')
        aff += new_aff
        num_new = new_aff.sum()
        bablos += 50 * num_new
        if num_new == 0:
            return bablos - 10000
        else:
            days_flag = True
    return bablos - 10000

In [17]:
# Let's check solution from step 1
n = 4039
aff = np.zeros(n)
aff[3057] = 1
simple_simulate_market(aff)

30200.0

Let's find the most profitable nodes.

In [18]:
one_node_value = np.zeros(n)
for i in tqdm(range(n)):
    test_aff = np.zeros(n)
    test_aff[i] = 1
    one_node_value[i] = simple_simulate_market(test_aff)

100%|██████████| 4039/4039 [01:00<00:00, 67.13it/s] 


In [19]:
sorted_one_values = np.argsort(one_node_value)
top35 = sorted_one_values[-35:]
top35

array([ 202, 2885,  356,  411, 2838, 3003, 3606, 3775,  594, 2788,  454,
       3988,  468,  443, 3939, 3989, 4013, 3993, 3985,  337,   93, 4011,
        167, 3991, 2558, 2217, 2450, 2113, 2439, 2548, 2528, 2263, 1505,
       3057, 1304], dtype=int64)

We will find the best pairs of 35 top elements.

In [20]:
fst = []
snd = []
value = []
for i in range(35):
    for j in range(35):
        if i < j:
            fst.append(top35[i])
            snd.append(top35[j])
            test_aff = np.zeros(n)
            test_aff[top35[i]] = 1
            test_aff[top35[j]] = 1
            value.append(simple_simulate_market(test_aff))
df = pd.DataFrame({'fst': fst, 'snd': snd, 'value': value})
df = df.sort_values(by=['value'], ascending=False)
df

,fst,snd,value
594,3057,1304,59000.0
242,3775,3057,52400.0
593,1505,1304,46700.0
591,2263,1304,41000.0
588,2528,1304,41000.0
...,...,...,...
252,594,3993,-1150.0
251,594,4013,-1150.0
250,594,3989,-1150.0
323,3988,4013,-1150.0


In [21]:
df.head(20)

,fst,snd,value
594,3057,1304,59000.0
242,3775,3057,52400.0
593,1505,1304,46700.0
591,2263,1304,41000.0
588,2528,1304,41000.0
584,2548,1304,40700.0
573,2113,1304,40100.0
579,2439,1304,40100.0
241,3775,1505,40100.0
587,2528,3057,39700.0


Vertices 1304, 3057 and 3775 interact well with each other.

Let's create the basis of the set of "best nodes" from nodes 1304, 3057 and 3775.

Next, we will add one node at a time using a greedy algorithm (as in step 2). We will use function simple_simulate_market. The algorithm stops when adding a vertex does not bring profit.

In [24]:
"""
"Доказательство этого утверждения предлагается читателю в качестве упражнения."
(c) Любой автор учебника, когда ему лень расписывать доказательство.
"""
# Result
good_nodes = [3775, 3057, 2263, 1304, 2568, 1084, 54, 2745, 167, 154]

In [26]:
aff = np.zeros(n)
for i in interest_nodes:
    aff[i] = 1
print(simple_simulate_market(aff))

128250.0


NOT BAD!!!

Now our goal is to arrange these vertices in the "real function" so that we do not go into negative on any of the days. 

By trial and error, we succeed!

In [27]:
simulate_market({0: [3775, 3057, 2263, 154], 12: [1304], 15: [2568, 1084], 16: [54], 19: [2745], 21: [167]})

(128250.0, 53, 3397.0)

### Step 4. "cheap" neighbors

The nodes selected by us are very good, but... too expensive.

Can we activate some good node by buying the "cheap" neighbors of the node?

Yes!

In [32]:
# It is enough for us to buy two friends of 3775.
G.degree[3775], np.ceil(G.degree[3775] * 0.18)

(8, 2.0)

In [33]:
for v in G.neighbors(3775):
    print(v, G.degree[v])

3953 3
3544 4
3547 12
3454 13
3496 18
3847 12
3660 5
3895 12


In [34]:
3 + 4 < 8

True

It is profitable for us to replace $3775$ with $3953$ and $3544$!

If we buy $3953$ and $3544$, then the useful node will be activated, while we will pay less.

In [35]:
simulate_market({0: [3953, 3544, 3057, 2263, 154], 12: [1304], 15: [2568, 1084], 16: [54], 19: [2745], 21: [167]})

(128500.0, 53, 3397.0)

Unfortunately, this trick does not work with other "good nodes".

Therefore, $128,500 is my best result.

Thanks for your attention!